# CeSci cadCAD model

This notebook contains a cadCAD model of the centralized science value flow as shown in the schema below. This model shows the value flow within science, from funding to publishing, and the value leakages throughout.

This third notebook adds the functionality introduces non-deterministic funding, where funding is disbursed with a certain probability to model the uncertainty researchers face when applying for grants.
![](opsci_baseline_stock&flow.png)

## Questions

## Mathematical Specification

Funding that reaches researchers:

$F_r = F_i \cdot (1-\alpha)$

where $\alpha \in [0,1]$ is the management cost weight, $F_i$ is the initial funding disbursed by the funding agency.

Value of created research assets:

$V_r = F_r \cdot (1-\epsilon)-c_p$

Salary of researchers:

$S_r = V_r \cdot \beta$

where $\beta\in[0,1]$ is the percentage of the research value that originated due to the direct work of the researcher.

where $\epsilon>0$ is the leakage due to innefficient resource allocation.

Value that ends up in journals:

$V_j = V_r + c_b$

where $c_p$ is the cost of publishing, $c_b$ is the cost of buying access to knowledge resource (in case a researcher buys from the journal).

## cadCAD Standard Notebook Layout

### 0. Dependencies

In [2]:
import pandas as pd
import numpy as np
from random import normalvariate, random
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor
from cadCAD import configs

### 1. State Variables

In [3]:
initial_state = {
    'funding_pool': 1000,
    'researcher_value': 100,
    'journal_value': 0,
    'losses': 0
}

### 2. System Parameters

In [4]:
system_params = {
    'funding_pool': [1000],
    'funding_round': [100],
    'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'beta': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'cost_publishing': [10],
    'cost_buying': [10],
    'probability_buying': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'probability_funding': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}

### 3. Policy Functions

Example:
Given initial funding 100, alpha 0.2, the funding that reaches the researchers is $100\cdot 0.8=80$. Next, given $\epsilon=0.1$, cost of publishing 10, the value of the research project is $80\cdot 0.9-10=62$.

In [21]:
def p_value_flow(params, substep, state_history, previous_state):
    if random() < params['probability_funding']:
        funding = params['funding_round']
        management_costs = funding * params['alpha']
        to_researcher = funding - management_costs
    else:
        funding = 0
        management_costs = 0
        to_researcher = 0
    losses = management_costs

    research_value = funding * (1-params['epsilon'])
    losses += to_researcher - research_value

    salary = research_value * params['beta']
    to_journal = research_value + params['cost_publishing']
    if random() < params['probability_buying']:
        salary = salary - params['cost_buying']
        to_journal += params['cost_buying']
    # losses = funding - to_journal
    return {'update_researcher_funding': to_researcher,
            'update_funding_pool': -funding,
            'update_journal': to_journal,
            'update_researcher_done': salary,
            'update_losses': losses}

### 4. State Update Functions

In [22]:
def s_researcher_value(params, substep, state_history, previous_state, policy_input):
    research_funding = policy_input['update_researcher_funding']
    research_salary = policy_input['update_researcher_done']
    research_value = previous_state['researcher_value']

    if research_salary == 0:
        updated_researcher_value = research_funding + research_value
        return 'researcher_value', updated_researcher_value
    else:
        updated_researcher_value = research_salary + research_value
        return 'researcher_value', updated_researcher_value

def s_journal_value(params, substep, state_history, previous_state, policy_input):
    to_journal = policy_input['update_journal']
    journal_value = previous_state['journal_value']

    updated_journal_value = to_journal + journal_value

    return 'journal_value', updated_journal_value

def s_funding_pool(params, substep, state_history, previous_state, policy_input):
    funding_pool = previous_state['funding_pool']
    updated_funding_pool = funding_pool + policy_input['update_funding_pool']
    return 'funding_pool', updated_funding_pool

def s_losses(params, substep, state_history, previous_state, policy_input):
    losses = previous_state['losses']
    updated_losses = losses + policy_input['update_losses']
    return 'losses', updated_losses

### 5. Partial State Update Blocks

In [23]:
partial_state_update_blocks = [
    {
        'policies': {
            'p_value_flow': p_value_flow
        },
        'variables': {
            'funding_pool': s_funding_pool,
            'researcher_value': s_researcher_value,
            'journal_value': s_journal_value,
            'losses': s_losses
        }
    }
]

### 6. Configuration

In [24]:
sim_config = config_sim({
    'N': 100,
    'T': range(10),
    'M': system_params
})

del configs[:]

In [25]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [26]:
print(system_params)

{'funding_pool': [1000], 'funding_round': [100], 'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'beta': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'cost_publishing': [10], 'cost_buying': [10], 'probability_buying': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'probability_funding': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}


### 7. Execution

In [27]:
exec_context = ExecutionContext()
configs = experiment.configs
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 9, 900, 4)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 9, 900, 4)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 34.79s


### 8. Analysis

In [15]:
pd.options.plotting.backend = "plotly"

In [28]:
df = pd.DataFrame(raw_result)
df

,funding_pool,researcher_value,journal_value,losses,simulation,subset,run,substep,timestep
0,1000,100.0,0.0,0.0,0,0,1,0,0
1,1000,100.0,10.0,0.0,0,0,1,1,1
2,1000,90.0,30.0,0.0,0,0,1,1,2
3,1000,90.0,40.0,0.0,0,0,1,1,3
4,1000,90.0,50.0,0.0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
9895,600,86.0,150.0,360.0,0,8,100,1,6
9896,500,85.0,180.0,450.0,0,8,100,1,7
9897,400,84.0,210.0,540.0,0,8,100,1,8
9898,300,83.0,240.0,630.0,0,8,100,1,9


In [29]:
df.plot(kind='line', x='timestep', y=['funding_pool','researcher_value', 'journal_value', 'losses'])

In [30]:
df.plot(kind='line', x='timestep', y=['funding_pool','researcher_value'])

In [31]:
df.plot(kind='line', x='timestep', y=['funding_pool', 'losses'])

In [32]:
df.plot(kind='line', x='timestep', y=['funding_pool', 'journal_value'])